In [ ]:
import networkx as nx
import csv

Create Nodes List

In [ ]:
teams = []
f = open("nhl_21_22.csv")
csv_read = csv.reader(f)
for row in csv_read:
  if not row[1] in teams:
    teams.append(row[1])
  if not row[3] in teams:
    teams.append(row[3])
  if len(teams) == 32:
    break
f.close()

Create edge dictionary

In [ ]:
def season_matches(file):
  matchups = {} #G, P, #games
  f = open(file)
  csv_read = csv.reader(f)
  for row in csv_read:
    goals1 = int(row[2])
    goals2 = int(row[4])
    points1 = 0
    points2 = 0
    if goals1 < goals2:
      points2 = 2
      if row[5] != "":
        points1 = 1
    elif goals2 < goals1:
      points1 = 2
      if row[5] != "":
        points2 = 1
    #Team 1 stats
    if f"{row[1]}-{row[3]}" not in matchups.keys():
      matchups[f"{row[1]}-{row[3]}"] = [goals1, points1, 1]
    else:
      matchups[f"{row[1]}-{row[3]}"][0] += goals1
      matchups[f"{row[1]}-{row[3]}"][1] += points1
      matchups[f"{row[1]}-{row[3]}"][2] += 1

    #Team 2 stats
    if f"{row[3]}-{row[1]}" not in matchups.keys():
      matchups[f"{row[3]}-{row[1]}"] = [goals2, points2, 1]
    else:
      matchups[f"{row[3]}-{row[1]}"][0] += goals2
      matchups[f"{row[3]}-{row[1]}"][1] += points2
      matchups[f"{row[3]}-{row[1]}"][2] += 1
  f.close()
  return matchups

matchups = season_matches("nhl_22_23.csv")
files = ["nhl_21_22.csv", "nhl_20_21.csv", "nhl_19_20.csv", "nhl_18_19.csv"]


i = 0
while i < len(files):
  tmp = season_matches(files[i])

  for pair in tmp.keys():
    matchups[pair][0] += (tmp[pair][0] * (0.8-(i*0.2)))
    matchups[pair][1] += (tmp[pair][1] * (0.8-(i*0.2)))
    matchups[pair][2] += (tmp[pair][2] * (0.8-(i*0.2)))
  i+=1

normalizer = -1
for pair in matchups.keys():
  normalizer = max(normalizer, matchups[pair][0]/matchups[pair][2])
edges = []
for pair in matchups.keys():
  tmp = pair.split("-")
  points = matchups[pair][1]/matchups[pair][2]
  goals = (matchups[pair][0]/matchups[pair][2]) / normalizer
  edges.append((tmp[1], tmp[0], points+goals))


Create Team network

In [ ]:
G = nx.DiGraph()
[G.add_node(k) for k in teams]
G.add_weighted_edges_from(edges)
nx.draw(G, with_labels=True)

Print Table

In [ ]:
pr1 = nx.pagerank(G)

temp = sorted(pr1, key=pr1.get, reverse=True)
for key in temp:
  print(f"{key}:  {round(pr1[key], 4)}")